In [ ]:
import itertools
import numpy as np

seed = 387
num_rounds = 100
num_types = 3

DEBUG = False
np.set_printoptions(precision=2, suppress=True)
rg = np.random.default_rng(seed)

airline_default_config = {
    'epLen': num_rounds,
    'f': np.asarray([1., 2.]),
    'A': np.transpose(np.asarray([[2, 3, 2], [3, 0, 1]])),
    'starting_state': np.asarray([7, 4, 4]),
    'P': np.asarray([[1/3, 1/3] for _ in range(num_rounds+1)])
}


In [ ]:
optimal_values = np.zeros((num_rounds,8,5,5))
for t in np.arange(num_rounds-1,-1,-1):
    if DEBUG: print(f'Timestep: {t}')
    for s in itertools.product(*[np.arange(airline_default_config['starting_state'][i]+1) for i in range(airline_default_config['starting_state'].shape[0])]):
        q_vals = []
        for a in [(0,0),(0,1),(1,0),(1,1)]:
            default_dim = tuple(np.append(t+1,s))
            next_vals = 0 if t+1 >= num_rounds else optimal_values[default_dim]
            if a == (0,0): # chose to reject both of them
                q_vals.append(next_vals)
            elif a == (0,1): # accept the second type
                temp_q = 0
                if np.min(s - airline_default_config['A'][:, 1]) >= 0: # checks if the allocation is feasible
                    new_dim = tuple(np.append(t+1, s - airline_default_config['A'][:, 1]))
                    temp_q += (1/3)*(2 + (0 if t+1 >= num_rounds else optimal_values[new_dim])) + (2/3)*next_vals
                else: # allocation is not feasible
                    temp_q = next_vals
                q_vals.append(temp_q)
            elif a == (1,0): # accepts the first type
                temp_q = 0
                if np.min(s - airline_default_config['A'][:, 0]) >= 0: # checks if the allocation is feasible
                    new_dim = tuple(np.append(t+1, s - airline_default_config['A'][:, 0]))
                    temp_q = (1/3)*(1 + (0 if t+1 >= num_rounds else optimal_values[new_dim]))+(2/3)*next_vals
                else: # allocation is not feasible
                    temp_q = next_vals
                q_vals.append(temp_q)
            elif a == (1,1): # try and accept both types
                temp_q = 0
                if np.min(s - airline_default_config['A'][:, 1]) >= 0: # checks if the allocation is feasible
                    new_dim = tuple(np.append(t+1, s - airline_default_config['A'][:, 1]))
                    temp_q += (1/3)*(2 + (0 if t+1 >= num_rounds else optimal_values[new_dim]))
                else:
                    temp_q + (1/3)*next_vals
                if np.min(s - airline_default_config['A'][:, 0]) >= 0: # checks if the allocation is feasible
                    new_dim = tuple(np.append(t+1, s - airline_default_config['A'][:, 0]))
                    temp_q += (1/3)*(1 + (0 if t+1 >= num_rounds else optimal_values[new_dim]))
                else:
                    temp_q += (1/3)*next_vals
                temp_q += (1/3)*next_vals
                q_vals.append(temp_q)
        dim = tuple(np.append(t,s))
#         print(np.max(q_vals))
        q_max = np.max(q_vals)
        optimal_values[dim]=q_max
        if DEBUG: print(f'State: {s}, Value: {q_max}')
        
print("V_star", optimal_values[0,7,4,4])